In [1]:
%load_ext autotime

In [2]:
# Importar librerias
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


import numpy as np

import zipfile
from datetime import date
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
from sklearn.metrics import classification_report


from hyperopt import hp, fmin, tpe, hp, STATUS_OK, Trials

from numpy.testing import assert_allclose

import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dropout, Dense, GlobalMaxPool1D, Conv1D, Flatten,  MaxPooling1D, Activation, GlobalMaxPooling1D, Bidirectional, GRU
from tensorflow.compat.v1.keras.layers import CuDNNGRU
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adadelta

import os
from pathlib import Path


C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Ed\anaconda3\envs\tf-gpu\lib\site-packages\tensor

time: 1min 10s


In [3]:
# Seed value
# Apparently you may use different seed values at each stage
seed_value= 0

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)

# 4. Set the `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

time: 10 ms


In [4]:
exp_name = '7_b'
folder = 'Resultados/' + exp_name
my_file = Path(folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(folder)
    


already exists
time: 148 ms


In [5]:
ch_folder = folder + '/Checkpoints'
my_file = Path(ch_folder)
if os.path.exists(my_file):
    print('already exists')
else:
    os.makedirs(ch_folder)

already exists
time: 146 ms


### Read datasets

In [6]:
#Importar los datasets
url_embeddings_average_individual = zipfile.ZipFile('../Data/average_bigram_topics_sentiment.zip')

embeddings_average_individual = pd.read_csv(url_embeddings_average_individual.open('average_bigram_topics_sentiment.csv'), index_col = 0)

embeddings_average_individual['Date'] =  pd.to_datetime(embeddings_average_individual['Date'], format='%Y-%m-%d')

embeddings_average_individual.reset_index(inplace=True)
embeddings_average_individual.fillna(0, inplace=True)
embeddings_average_individual.tail(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,Top,Date,Label,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,sentiment
49649,0.000830,0.050684,-0.078711,0.069385,-0.120630,0.095508,0.036621,0.002893,0.096631,0.027344,0.134375,-0.150244,-0.100488,0.123291,0.020605,0.104590,0.079565,0.159546,-0.014038,-0.019336,0.005188,-0.091040,0.018677,0.007471,-0.058362,-0.175696,0.052246,-0.047949,0.057214,-0.001563,-0.074683,-0.089453,-0.136575,-0.035522,-0.044775,-0.084277,-0.087115,0.092236,-0.074023,0.171582,0.055505,-0.031738,0.079736,0.000439,0.062964,-0.156643,0.037891,0.053223,-0.184277,0.015015,-0.042213,0.110443,-0.159143,0.075684,0.050726,-0.124219,-0.053876,0.066498,0.046753,-0.142627,0.001953,0.062012,0.155615,-0.119690,0.023047,0.107422,-0.012073,0.052051,-0.038714,0.096625,0.032764,-0.105762,0.075488,-0.013354,-0.159668,-0.022070,-0.127051,-0.002856,0.020410,0.096093,0.051074,-0.160498,-0.044824,0.037259,0.032520,0.038721,-0.170763,0.044312,-0.043982,-0.170801,0.001929,-0.107178,-0.018710,-0.023686,0.083203,-0.066431,0.240039,0.015967,0.080176,0.083057,0.085913,0.160815,-0.024756,0.089258,0.134863,-0.183105,0.093701,-0.032422,-0.030859,-0.171924,-0.070312,-0.000861,0.073975,-0.145459,0.006323,-0.124481,-0.067749,-0.072168,0.152429,0.090430,-0.165698,0.082666,-0.073535,-0.021973,-0.027893,-0.214258,0.093848,0.023438,0.018594,0.048389,0.057397,0.013354,-0.235205,-0.159955,-0.205176,-0.246289,-0.039508,0.218457,-0.163428,0.123743,0.073145,-0.007524,-0.051318,0.064874,0.069727,-0.036743,-0.072217,-0.016357,-0.172266,-0.006573,0.173486,-0.005371,-0.003857,0.003418,-0.031494,-0.094775,0.090625,-0.196759,-0.032446,-0.116797,0.097949,0.073145,0.187024,-0.023914,-0.021729,-0.093010,0.054492,0.035657,-0.014432,-0.093689,0.002148,-0.012500,-0.125958,-0.115039,-0.030200,-0.080908,0.124854,-0.017725,0.061914,0.010327,-0.130652,-0.037939,0.100732,-0.101514,0.065918,0.064258,-0.105762,0.006689,0.008521,0.134717,-0.051904,0.046875,0.099512,0.149072,-0.068091,-0.006396,0.045746,-0.036450,-0.158325,-0.030737,0.049097,0.045508,-0.149609,0.038666,0.093359,0.014551,-0.016589,-0.057178,0.11460,-0.064404,-0.166699,0.026172,-0.048755,-0.047766,-0.108203,-0.067676,0.05293,0.105859,-0.162329,-0.151929,0.009521,-0.007690,0.120850,-0.032813,-0.003711,-0.147778,0.150977,-0.049023,-0.017871,0.169336,0.044727,-0.014014,-0.022339,-0.033347,0.001709,-0.017822,-0.022852,0.067737,0.113086,-0.056299,0.023663,-0.058374,0.073743,-0.199902,0.022217,0.091113,-0.015039,0.064966,-0.006104,-0.209302,0.096289,0.058203,0.109180,-0.001855,-0.074133,-0.156067,-0.010223,0.082349,-0.107092,0.155078,-0.066010,0.070361,-0.042505,0.131006,0.076392,-0.072461,-0.046094,0.154224,-0.020947,-0.030371,0.039795,0.204150,0.089087,-0.191699,-0.013916,-0.139600,-0.072852,-0.053857,-0.142529,0.057910,0.062061,-0.052393,0.167090,0.058887,-0.076538,-0.095724,-0.230847,0.036865,0.106544,0.180566,0.070203,0.108008,-0.031592,-0.192676,-

time: 13.6 s


### Embedding Promedio

In [7]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2013-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

time: 170 ms


In [8]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual[embeddings_average_individual['Date']<='2014-12-31']
print(data['Label'].mean())
data.head(1)

0.5418076004775169


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,Top,Date,Label,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,sentiment
9443,-0.063354,0.015055,-0.127116,0.055664,-0.07666,-0.111328,-0.004476,-0.130859,0.148763,0.015381,0.095835,-0.082113,0.023112,0.041992,-0.063232,0.176595,0.047791,0.068827,0.000814,0.072917,0.039714,0.004801,-0.130697,-0.138672,0.013018,0.010295,-0.103597,0.046224,0.167643,0.028768,-0.177083,0.079997,-0.026204,-0.10555,-0.044434,0.032288,-0.023071,0.133301,0.076497,0.03304,0.009018,-0.049886,0.048991,-0.166972,-0.024577,-0.147461,0.059896,0.010742,0.14091,0.0271,-0.012817,0.015625,0.062581,-0.09082,0.042013,-0.231608,0.029012,-0.036865,0.06665,0.036418,0.096842,-0.008626,-0.129781,-0.075521,-0.066447,-0.120361,-0.009572,0.056966,0.158529,0.16154,-0.03658,-0.037516,-0.003642,-0.131917,-0.201009,-0.108358,-0.025553,0.015381,-0.040436,0.02002,-0.031291,-0.091309,0.06901,-0.016602,0.061971,-0.024089,-0.119954,0.102397,-0.027995,0.075684,0.003418,-0.054688,-0.162069,-0.056152,-0.052409,-0.097493,0.134684,0.032633,-0.05717,0.018066,0.025065,-0.104533,0.061686,0.041504,-0.013387,-0.01062,0.011882,0.077637,0.152445,-0.071828,-0.009847,0.015299,-0.055908,0.00061,-0.220062,-0.024801,0.030268,0.121989,0.059082,0.096293,-0.104167,-0.1014,-0.065104,0.127686,-0.031738,-0.002238,-0.182292,-0.149495,0.057251,0.097656,0.085836,-0.180339,-0.055257,0.009542,0.064941,-0.265706,0.073486,-0.054606,-0.061198,0.127223,0.051229,-0.170736,-0.05245,0.066813,0.055216,0.15918,-0.208008,-0.182129,-0.025879,-0.073324,-0.01237,0.035156,-0.051432,0.126363,-0.091227,-0.164185,-0.067464,-0.015137,-0.085531,0.055094,-0.141439,0.014974,-0.056966,-0.028809,-0.027262,-0.048706,0.064209,-0.163574,-0.007894,-0.114258,-0.158203,-0.058512,0.029785,-0.056844,-0.053385,0.063354,0.063477,0.014608,-0.142253,0.080078,-0.045736,-0.042033,0.046987,-0.000651,-0.062826,-0.066081,-0.04777,0.038656,0.033529,0.038778,0.082642,-0.078125,0.050781,0.036214,-0.152832,0.25472,-0.013672,-0.063721,-0.125651,-0.277344,-0.002991,0.078242,-0.183431,-0.183838,0.157064,0.077474,-0.017049,0.025187,-0.011759,-0.11084,-0.094686,0.106974,-0.17806,0.056348,-0.158773,0.05306,0.239258,0.140951,-0.034424,-0.002797,-0.057048,-0.156576,-0.014974,0.191121,0.015574,-0.012594,-0.008545,-0.000814,-0.024333,0.123698,0.172526,-0.055766,-0.105957,-0.061605,0.05485,0.08846,0.043294,0.020447,0.105347,-0.008301,-0.0118,-0.017853,-0.113118,-0.303304,-0.09789,0.066203,0.056315,-0.090515,-0.113118,0.047811,-0.075928,-0.015381,0.018717,0.018717,-0.019368,-0.086304,-0.051595,0.061526,0.036438,-0.060476,0.118571,0.104329,-0.006226,0.249349,0.008057,0.076986,0.111287,-0.207357,-0.124064,-0.05837,0.099202,0.141357,0.039876,0.09493,-0.094238,0.105469,-0.138835,-0.245687,-0.046631,0.069092,0.081706,0.022705,-0.067546,0.114583,0.088623,0.109782,-0.175354,0.104126,-0.074707,0.083984,0.067464,0.001546,-0.053955,0.056478,0.055176,-0.096842,-0.020833,0

time: 828 ms


In [9]:
training = data[:num_training]
testing = data[num_training:]

# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, x_train.shape[1])
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, x_train.shape[1])
reshape_x_test.shape

(6294, 1, 311)

time: 211 ms


### Definir espacio de busqueda

In [10]:
space = {
    'units1': hp.choice('units1', [8, 16, 32, 64, 128, 256, 512]),
    'units2': hp.choice('units2', [8, 16, 32, 64, 128, 256, 512]),
                 
    'dropout1': hp.choice('dropout1', [0.1,0.2,0.3, 0.4]),
    
    'batch_size' : hp.choice('batch_size', [16,64,128,256]),
    
    'nb_epochs' : hp.choice('nb_epochs', [50]),

    'optimizer':  hp.choice('optimizer', [ 'adam','adadelta']),   
    'activation': hp.choice('activation', [ 'relu','softmax']), 
    }

time: 510 ms


### Definir busqueda bayesiana

In [15]:
#Objective function that hyperopt will minimize
from timeit import default_timer as timer

def objective(params):
    
#     import ml_metrics

    
    start = timer()
    print ('Params testing: ', params)
    print ('\n ')
  
    model = Sequential()
    model.add(Conv1D(128, 1, activation='relu'))
    model.add(GlobalMaxPool1D())
    model.add(Dense(params['units1'], activation = params['activation']))
    model.add(Dense(1, activation = 'sigmoid'))
    model.compile(optimizer=params['optimizer'], loss='binary_crossentropy', metrics=['accuracy'])



    logdir = "Resultados\\" + exp_name +"\\logs\\model"

    tensor_board = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch = 100000000)

    
    #includes the call back object
    model.fit(reshape_x_train, y_train, epochs=params['nb_epochs'], batch_size=params['batch_size'],
                verbose = 0, validation_data=(reshape_x_test, y_test),callbacks=[tensor_board])
     
    #predict the test set 
    ypred = model.predict_proba(reshape_x_test)
    testing_cp = testing.copy()
    testing_cp['Prob'] = ypred
    testing_cp['Prob_dia'] = testing_cp['Prob'].groupby(testing_cp['Date']).transform('mean')
    testing_cp['Prediction'] = 0
    testing_cp.loc[testing_cp['Prob_dia']>0.5, 'Prediction'] = 1
    testing_cp.drop_duplicates(subset=['Date','Prediction','Label'], inplace=True)
    
    acc = accuracy_score(testing_cp.Label, testing_cp.Prediction)
    
    run_time = timer() - start
    
    # Write to the csv file ('a' means append)
    of_connection = open(out_file, 'a')
    writer = csv.writer(of_connection)
    writer.writerow([-acc, params, run_time])
    of_connection.close()
    
    
    print('Test accuracy:', acc)
 
    return {'loss': -acc, 'status': STATUS_OK, 'train_time': run_time,}



time: 9 ms


### Almacenar resultados de cada iteración

In [16]:
from hyperopt import tpe

tpe_algorithm = tpe.suggest

time: 999 µs


In [17]:
import csv

from hyperopt import Trials

bayes_trials = Trials()

# File to save first results
out_file = folder + '/gbm_results.csv'
of_connection = open(out_file, 'w')

writer = csv.writer(of_connection)

# Write the headers to the file
writer.writerow(['loss', 'params', 'time'])
of_connection.close()

time: 405 ms


### Lanzar optimización

In [ ]:
# Run optimization
best = fmin(fn = objective, space = space, algo = tpe.suggest, 
            max_evals = 2, trials = bayes_trials,
            verbose = 1, rstate= np.random.RandomState(50))


Params testing:                                                                                                        
{'activation': 'relu', 'batch_size': 64, 'dropout1': 0.2, 'nb_epochs': 50, 'optimizer': 'adam', 'units1': 256, 'units2': 32}
  0%|                                                                            | 0/2 [00:00<?, ?trial/s, best loss=?]

### Exportar bayesiana, por si quisiera retomar donde queda

In [ ]:
import pickle
pickle.dump(bayes_trials, open(folder + '/trials.pkl', 'wb'))

### Leer mejores parametros

In [ ]:
# Sort the trials with lowest loss (highest AUC) first
bayes_trials_results  = sorted(bayes_trials.results, key = lambda x: x['loss'])
bayes_trials_results [:2]

In [ ]:
results = pd.read_csv(folder + '/gbm_results.csv')

# Sort with best scores on top and reset index for slicing
results.sort_values('loss', ascending = True, inplace = True)
results.reset_index(inplace = True, drop = True)
results.head()

In [ ]:
import ast

# Convert from a string to a dictionary
ast.literal_eval(results.loc[0, 'params'])

In [ ]:
# Extract the ideal number of estimators and hyperparameters
best_bayes_params = ast.literal_eval(results.loc[0, 'params']).copy()
best_bayes_params

### Definir datasets de testeo

In [ ]:
# Selecciono la fecha para la cual hago el corte de train y test
training_end = pd.to_datetime("2014-12-31")
num_training = len(embeddings_average_individual[(embeddings_average_individual["Date"]) <= training_end])

In [ ]:
# Selecciono el archivo con el que se corre el modelo
data = embeddings_average_individual
data.head(1)

In [ ]:
training = data[:num_training]
testing = data[num_training:]

# Se separa en train y test
x_train = data.drop(["Top","Label", "Date"], axis=1)[:num_training]
x_test = data.drop(["Top",'Label', 'Date'], axis=1)[num_training:]
y_train = data["Label"].values[:num_training]
y_test = data["Label"].values[num_training:]


x_train_array = x_train.to_numpy()
reshape_x_train = x_train_array.reshape(len(x_train), 1, x_train.shape[1])
reshape_x_train.shape

x_test_array = x_test.to_numpy()
reshape_x_test = x_test_array.reshape(len(x_test), 1, x_train.shape[1])
reshape_x_test.shape

In [ ]:
model = Sequential()
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(GlobalMaxPool1D())
model.add(Dense(best_bayes_params['units1'], activation=best_bayes_params['activation']))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy',         metrics=['accuracy'])


# define the checkpoint
filepath= ch_folder + "/word2vec-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:

logdir = "Resultados\\" + exp_name +"\\logs\\model"


tensor_board = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1, profile_batch = 100000000)


In [ ]:
# fit the model
model.fit(reshape_x_train, y_train,
          epochs=best_bayes_params['nb_epochs'], 
          batch_size=best_bayes_params['batch_size'], callbacks=[tensor_board])


model.save(folder + '/keras_model.h5')
model.summary()

In [ ]:
loss, accuracy = model.evaluate(reshape_x_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(reshape_x_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


In [ ]:
# evaluate the model
ypred = model.predict_proba(reshape_x_test)
ypred

In [ ]:
testing_cp = testing.copy()
testing_cp['Prob'] = ypred
testing_cp['Prob_dia'] = testing_cp['Prob'].groupby(testing_cp['Date']).transform('mean')
testing_cp['Prediction'] = 0
testing_cp.loc[testing_cp['Prob_dia']> 0.5, 'Prediction'] = 1
testing_cp.drop_duplicates(subset=['Date','Prediction','Label'], inplace=True)
testing_cp.head(1)

print('Numero de días en testing: ', testing['Date'].nunique())
print('Numero de registros en testing: ', testing_cp['Date'].nunique())

In [ ]:
testing_cp['Label'].sum()

In [ ]:
display(testing_cp['Label'].value_counts(),
        testing_cp['Prediction'].value_counts() )

In [ ]:
target_names = ['class 0', 'class 1']
print(classification_report(testing_cp.Label, testing_cp.Prediction, target_names=target_names))

In [ ]:

# Creates a confusion matrix
cm = confusion_matrix(testing_cp.Label, testing_cp.Prediction) 

# Transform to df for easier plotting
cm_df = pd.DataFrame(cm,
                     index = ['Sube','Baja'], 
                     columns = ['Sube','Baja'])
plt.figure(figsize=(7,5))
sns.heatmap(cm_df, annot=True, cmap="Greens", fmt='g')
plt.title('Neuronal Network \nAccuracy:{0:.3f}'.format(accuracy_score(testing_cp.Label, testing_cp.Prediction)))
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()


In [ ]:
%%javascript
Jupyter.notebook.save_checkpoint()

In [ ]:
from time import sleep
sleep(60)

In [ ]:
from shutil import copyfile
copyfile('RNN_Model_Base_GPU_m2.ipynb', folder + '/RNN_Model_Base.ipynb' )